In [20]:
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification, make_regression

from minimizers.minimize import GeneralLossMinimizer, CustomLossClassifier, CustomLossRegressor


In [21]:
seed = 0

---
### Standard classification with  `GeneralLossMinimizer`

In [22]:
X, y = make_classification(
    n_classes=2, 
    n_samples=1000, 
    n_features=10, 
    flip_y=0., 
    n_clusters_per_class=1, 
    random_state=seed,
)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)
del X, y

In [23]:
mod = Pipeline([
    ('sc', StandardScaler()),
    ('clf', GeneralLossMinimizer(random_state=seed)),
])
mod[-1].set_estimator_type('classifier')
mod[-1].set_multi_output(False)

GeneralLossMinimizer(random_state=0)

In [24]:
mod.fit(X_train, y_train)

Pipeline(steps=[('sc', StandardScaler()),
                ('clf', GeneralLossMinimizer(random_state=0))])

In [25]:
y_hat = mod.predict(X_train).round()
(y_hat==y_train).mean()

0.9226666666666666

In [26]:
y_hat = mod.predict(X_test).round()
(y_hat==y_test).mean()

0.92

---
### Standard regression with  `GeneralLossMinimizer`

In [27]:
X, y = make_regression(
    n_targets=1, 
    n_samples=1000, 
    n_features=10, 
    random_state=seed,
)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)
del X, y

In [28]:
mod = Pipeline([
    ('sc', StandardScaler()),
    ('clf', GeneralLossMinimizer(random_state=seed)),
])
mod[-1].set_estimator_type('regressor')
mod[-1].set_multi_output(False)

GeneralLossMinimizer(random_state=0)

In [29]:
mod.fit(X_train, y_train)

Pipeline(steps=[('sc', StandardScaler()),
                ('clf', GeneralLossMinimizer(random_state=0))])

In [30]:
y_hat = mod.predict(X_train).round()
((y_train-y_hat)**2).mean()**0.5

0.29203808573391965

In [31]:
y_hat = mod.predict(X_test).round()
((y_test-y_hat)**2).mean()**0.5

0.2839428616952984

---
### Binary classification with `CustomLossClassifier` 

In [32]:
X, y = make_classification(
    n_classes=2, 
    n_samples=1000, 
    n_features=10, 
    flip_y=0., 
    n_clusters_per_class=1, 
    random_state=seed,
)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)
del X, y

In [33]:
mod = Pipeline([
    ('sc', StandardScaler()),
    ('clf', CustomLossClassifier(random_state=seed)),
])

In [34]:
mod.fit(X_train, y_train)

Pipeline(steps=[('sc', StandardScaler()),
                ('clf', CustomLossClassifier(random_state=0))])

In [35]:
y_hat = mod.predict(X_train)

In [36]:
y_hat = mod.predict(X_train).argmax(1)
(y_hat==y_train).mean()

0.9226666666666666

In [37]:
y_hat = mod.predict(X_test).argmax(1)
(y_hat==y_test).mean()

0.92

---
### Multi-categorical classification with `CustomLossClassifier`

In [38]:
X, y = make_classification(
    n_classes=3, 
    n_samples=1000, 
    n_features=10, 
    flip_y=0., 
    n_clusters_per_class=1, 
    random_state=seed,
)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)
del X, y

In [39]:
mod = Pipeline([
    ('sc', StandardScaler()),
    ('clf', CustomLossClassifier(random_state=seed)),
])

In [40]:
mod.fit(X_train, y_train)

Pipeline(steps=[('sc', StandardScaler()),
                ('clf', CustomLossClassifier(random_state=0))])

In [41]:
y_hat = mod.predict(X_train).argmax(1)
(y_hat==y_train).mean()

0.86

In [42]:
y_hat = mod.predict(X_test).argmax(1)
(y_hat==y_test).mean()

0.872

---
### Single output `CustomLossRegressor`

In [43]:
X, y = make_regression(
    n_targets=1, 
    n_samples=1000, 
    n_features=10, 
    random_state=seed,
)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)
del X, y

In [44]:
mod = Pipeline([
    ('sc', StandardScaler()),
    ('clf', CustomLossRegressor(random_state=seed)),
])

In [45]:
mod.fit(X_train, y_train)

Pipeline(steps=[('sc', StandardScaler()),
                ('clf', CustomLossRegressor(random_state=0))])

In [46]:
y_hat = mod.predict(X_train).squeeze()
((y_train-y_hat)**2).mean()**0.5

4.9928711244253936e-05

In [47]:
y_hat = mod.predict(X_test).squeeze()
((y_test-y_hat)**2).mean()**0.5

5.395171147134042e-05

---
### Multi-output regression with `CustomLossRegressor`

In [48]:
X, y = make_regression(
    n_targets=2, 
    n_samples=1000, 
    n_features=10, 
    random_state=seed,
)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)
del X, y

In [49]:
mod = Pipeline([
    ('sc', StandardScaler()),
    ('clf', CustomLossRegressor(random_state=seed)),
])

In [50]:
mod.fit(X_train, y_train)

Pipeline(steps=[('sc', StandardScaler()),
                ('clf', CustomLossRegressor(random_state=0))])

In [51]:
y_hat = mod.predict(X_train)
((y_train-y_hat)**2).mean(1).mean()**0.5

3.3306480815632496e-05

In [52]:
y_hat = mod.predict(X_test)
((y_test-y_hat)**2).mean(1).mean()**0.5

3.4671141657123606e-05

---
### Squared hinge loss with  `GeneralLossMinimizer`

In [53]:
def squared_hinge_loss(y, y_hat):

    zeros = np.zeros_like(y)
    margin = 1 - (y*y_hat)

    return np.c_[zeros, margin].max(1) ** 2

def linear_link(X, b):
    
    return X.dot(b)


In [54]:
X, y = make_classification(
    n_classes=2, 
    n_samples=1000, 
    n_features=10, 
    flip_y=0., 
    n_clusters_per_class=1, 
    random_state=seed,
)
y = np.where(y==1, 1, -1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed)
del X, y

In [55]:
mod = Pipeline([
    ('sc', StandardScaler(
    )),
    ('clf', GeneralLossMinimizer(
        random_state=seed, 
        loss_fn=squared_hinge_loss, 
        link_fn=linear_link,
    )),
])

In [56]:
mod.fit(X_train, y_train)

Pipeline(steps=[('sc', StandardScaler()),
                ('clf',
                 GeneralLossMinimizer(link_fn=<function linear_link at 0x12278cca0>,
                                      loss_fn=<function squared_hinge_loss at 0x12278d360>,
                                      random_state=0))])

In [57]:
y_hat = mod.predict(X_train)
y_hat = np.where(y_hat>0, 1, -1)
(y_hat==y_train).mean()

0.9226666666666666

In [58]:
y_hat = mod.predict(X_test)
y_hat = np.where(y_hat>0, 1, -1)
(y_hat==y_test).mean()

0.928

---